In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload

In [3]:
from convert_pc_to_usd import load_GDP,return_usd

In [4]:
GDP_file = 'C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\summary_all_data.xlsx'

In [5]:
GDP2 = load_GDP(GDP_file)

In [6]:
resilience_folder = "C:\\Users\\WB451855\OneDrive - WBG\\!infra_needs_data\\resilience\\"

In [9]:
inv_needs = pd.read_excel(resilience_folder+"power robcosts_Julie.xlsx","power_investment_needs")

In [26]:
extra_cost_exposure = pd.read_excel(resilience_folder+"power robcosts_Julie.xlsx","cost_increase_exposure")
extra_cost_all = pd.read_excel(resilience_folder+"power robcosts_Julie.xlsx","cost_increase_all")

In [12]:
inv_needs.sample(5)

,Variable,Model,Region,Scenario,2015,2020,2025,2030
177,EnergyInv_gdp,WITCH-GLOBIOM,R5REF,2C,0.032554,0.028219,0.025917,0.015186
31,EnergyInv_gdp_increasedstandards,MESSAGEix-GLOBIOM,R5ASIA,NDC,0.016304,0.013576,0.008371,0.005749
0,EnergyInv_gdp_increasedstandards,AIM/CGE,R5ASIA,2C,0.016947,0.014678,0.031675,0.035293
153,EnergyInv_gdp,REMIND-MAgPIE,R5LAM,2C,0.015933,0.016342,0.016534,0.021966
30,EnergyInv_gdp_increasedstandards,MESSAGEix-GLOBIOM,R5ASIA,2C,0.016304,0.013576,0.009487,0.007417


In [19]:
inv_needs_base = inv_needs[inv_needs.Variable=="EnergyInv_gdp"]

In [20]:
inv_needs_base = inv_needs_base.set_index(["Model","Region","Scenario"]).drop("Variable",axis=1)
inv_needs_base.columns.name='year'
inv_needs_base = inv_needs_base.stack().to_frame().rename(columns={0:"divided_by_gdp"})

In [27]:
extra_cost_all = extra_cost_all.set_index(["Model","Region","Scenario"]).drop("Discounted",axis=1)
extra_cost_all.columns.name='year'
extra_cost_all = extra_cost_all.stack().to_frame().rename(columns={0:"cost_increase_all"})

In [34]:
extra_cost_exposure = extra_cost_exposure.set_index(["Model","Region","Scenario"]).drop("Discounted",axis=1)
extra_cost_exposure.columns.name='year'
extra_cost_exposure = extra_cost_exposure.stack().to_frame().rename(columns={0:"cost_increase"})

In [38]:
all_res = inv_needs_base.reset_index().merge(extra_cost_all.reset_index(),on=["Model","Region","Scenario","year"]).merge(extra_cost_exposure.reset_index(),on=["Model","Region","Scenario","year"])

In [40]:
all_res['divided_by_gdp_res'] = all_res['divided_by_gdp']*(1+all_res['cost_increase'])
all_res['divided_by_gdp_res_all'] = all_res['divided_by_gdp']*(1+all_res['cost_increase_all'])

In [41]:
all_res['cost_diff'] = all_res['divided_by_gdp_res']-all_res['divided_by_gdp']
all_res['cost_diff_all'] = all_res['divided_by_gdp_res_all']-all_res['divided_by_gdp']

In [43]:
all_res_av = all_res.set_index(["Model","Region","Scenario",'year']).mean(level=(["Model","Region","Scenario"]))

In [44]:
all_res_av.reset_index().Region.unique()

array(['R5ASIA', 'R5LAM', 'R5MAF', 'R5OECD90+EU', 'R5REF'], dtype=object)

In [45]:
all_res_av = all_res_av.reset_index()

all_res_av["region"] = all_res_av.Region.replace({"R5MAF":"Africa",\
                                      "R5LAM":"Latin America and Caribbean",\
                                      "R5REF":"Eastern Europe and Central Asia",\
                                      "R5ASIA":"Asia",\
                                      "R5OECD90+EU":"OECD"})

In [46]:
all_res_av_usd = return_usd(all_res_av,'region','cost_diff',["Model","Scenario"],GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [49]:
all_res_av_usd.set_index(['region',"Model","Scenario"]).sum(level=["Model","Scenario"]).cost_bUSD_disc_annual.describe()

count    18.000000
mean     16.420776
std       5.511063
min       8.989644
25%      13.089095
50%      15.832537
75%      19.561188
max      27.549049
Name: cost_bUSD_disc_annual, dtype: float64

In [50]:
all_res_av_usd_tot = all_res_av_usd.set_index(['region',"Model","Scenario"]).sum(level=["Model","Scenario"])

In [51]:
all_res_av_usd_tot.loc[all_res_av_usd_tot['cost_bUSD_disc_annual'].idxmin()]

cost_bUSD_disc           134.844660
cost_bUSD_disc_annual      8.989644
Name: (MESSAGEix-GLOBIOM, CPol), dtype: float64

In [52]:
all_res_av_usd_tot.loc[all_res_av_usd_tot['cost_bUSD_disc_annual'].idxmax()]

cost_bUSD_disc           413.235736
cost_bUSD_disc_annual     27.549049
Name: (REMIND-MAgPIE, 2C), dtype: float64

In [53]:
all_res_av_usd_all = return_usd(all_res_av,'region','cost_diff_all',["Model","Scenario"],GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [54]:
all_res_av_usd_all_tot = all_res_av_usd_all.set_index(['region',"Model","Scenario"]).sum(level=["Model","Scenario"])

In [55]:
all_res_av_usd_all_tot.cost_bUSD_disc_annual.describe()

count     18.000000
mean     178.801973
std       60.102166
min       95.896689
25%      145.169809
50%      166.015545
75%      227.623981
max      296.489310
Name: cost_bUSD_disc_annual, dtype: float64

In [56]:
all_res_av[['cost_increase','cost_increase_all']].describe()

,cost_increase,cost_increase_all
count,90.000000,90.000000
mean,0.030363,0.294836
std,0.018624,0.022412
min,0.001410,0.198443
25%,0.022454,0.283698
50%,0.027092,0.295448
75%,0.038332,0.309101
max,0.067434,0.336481


In [57]:
all_res_av[['cost_increase','cost_increase_all']].mean()

cost_increase        0.030363
cost_increase_all    0.294836
dtype: float64